In [5]:
from dask import dataframe as dd
import dask_ml
from dask_ml.preprocessing import Categorizer, DummyEncoder
from sklearn.pipeline import Pipeline, make_pipeline

In [6]:
ddf=dd.read_csv(
    "https://gist.githubusercontent.com/geraldwal/b5a83f4c670abe0a662abce558e5d433/raw/bce4bbfc63355606e4503964e25798b5d2190b9b/hour%2520-%2520Python%2520Bike%2520Sharing",
    sep=",",
    parse_dates=["dteday"])

In [7]:
ddf = ddf.rename(
    columns={
        "weathersit": "weather",
        "mnth": "month",
        "hr": "hour",
        "hum": "humidity",
        "cnt": "count",
        "yr": "year",
    })

In [9]:
cats= ["season","month","hour","holiday","weekday","workingday","weather"]

pipeline = make_pipeline(Categorizer(columns=cats), DummyEncoder(columns=cats))
ddf_onehot=pipeline.fit_transform(ddf)


In [10]:
def split_data(dataset, Target):
    X = dataset.loc[:, dataset.columns != Target]
    y = dataset.loc[:, Target]
    train_size = int(len(dataset) * 0.875)
    X_train, X_test, y_train, y_test = (
        X[0:train_size],
        X[train_size : len(dataset)],
        y[0:train_size],
        y[train_size : len(dataset)],
    )
    return X_train, X_test, y_train, y_test

In [12]:
x_train_reg, x_test_reg, y_train_reg, y_test_reg = split_data(ddf_onehot, "registered")
x_train_casual, x_test_casual, y_train_casual, y_test_casual = split_data(ddf_onehot, "casual")
x_train_count, x_test_count, y_train_count, y_test_count = split_data(ddf_onehot, "count")

NotImplementedError: 'DataFrame.iloc' only supports selecting columns. It must be used like 'df.iloc[:, column_indexer]'.